In [1]:
from pathlib import Path
RAW = Path("/Users/liruiling/Desktop/COCHA/raw")
RAW.exists(), list(RAW.glob("*.pkl"))[:3]

(True,
 [PosixPath('/Users/liruiling/Desktop/COCHA/raw/1950-vocab.pkl'),
  PosixPath('/Users/liruiling/Desktop/COCHA/raw/1980-vocab.pkl'),
  PosixPath('/Users/liruiling/Desktop/COCHA/raw/1900-vocab.pkl')])